In [2]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import os

In [2]:
BITQUERY_API_TOKEN = 'BQY0mSI2IadezoknHO3JMzexNwCpugm5'
import config
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.GOOGLE_SERVICE_AUTH_JSON

In [ ]:
Query = """query ($network: EthereumNetwork!, $address: String!) {
  ethereum(network: $network) {
    address(address: {is: $address}) {
      smartContract {
        attributes {
          name
          type
          address {
            address
            annotation
          }
          value
        }
      }
    }
  }
}
"""

In [ ]:
r = requests.post(endpoint, json={"query": Query})
print(json.dumps(r.json(), indent=2))

In [3]:
def download_bigquery_price_mainnet(
    contract_address, date_begin, date_end, block_start
):
    """
    Internal function to query Google Bigquery for the swap history of a Uniswap v3 pool between two dates starting from a particular block from Ethereum Mainnet.
    Use GetPoolData.get_pool_data_bigquery which| preprocesses the data in order to conduct simualtions with the Active Strategy Framework.
    """

    from google.cloud import bigquery

    client = bigquery.Client()

    query = (
        """
            SELECT *
            FROM blockchain-etl.ethereum_uniswap.UniswapV3Pool_event_Swap
            where contract_address = lower('"""
        + contract_address.lower()
        + """') and
              block_timestamp >= '"""
        + str(date_begin)
        + """' and block_timestamp <= '"""
        + str(date_end)
        + """' and block_number >= """
        + str(block_start)
        + """
            """
    )
    query_job = client.query(query)  # Make an API request.
    return query_job.to_dataframe(create_bqstorage_client=False)

In [4]:
download_bigquery_price_mainnet('0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', '2022-04-30', '2022-05-05', 100)

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/imposing-ring-349623/jobs?prettyPrint=false: Access Denied: Project imposing-ring-349623: User does not have bigquery.jobs.create permission in project imposing-ring-349623.

Location: None
Job ID: 37e9eec6-9d05-441f-ba3e-1dd76c08a9cd


In [ ]:
def get_pool_data_bigquery(
    contract_address,
    date_begin,
    date_end,
    decimals_0,
    decimals_1,
    block_start=0,
):

    """
    Queries Google Bigquery for the swap history of Uniswap v3 pool between two dates starting from a particular block of Ethereum Mainnet.
    Preprocesses data to have decimal adjusted amounts and liquidity values.
    """

    resulting_data = download_bigquery_price_mainnet(
    contract_address.lower(), date_begin, date_end, block_start


    DECIMAL_ADJ = 10 ** (decimals_1 - decimals_0)
    resulting_data["sqrtPriceX96_float"] = resulting_data["sqrtPriceX96"].astype(float)
    resulting_data["quotePrice"] = (
        (resulting_data["sqrtPriceX96_float"] / 2**96) ** 2
    ) / DECIMAL_ADJ
    resulting_data["block_date"] = pd.to_datetime(resulting_data["block_timestamp"])
    resulting_data = resulting_data.set_index("block_date", drop=False).sort_index()

    resulting_data["tick_swap"] = resulting_data["tick"].astype(int)
    resulting_data["amount0"] = resulting_data["amount0"].astype(float)
    resulting_data["amount1"] = resulting_data["amount1"].astype(float)
    resulting_data["amount0_adj"] = (
        resulting_data["amount0"].astype(float) / 10**decimals_0
    )
    resulting_data["amount1_adj"] = (
        resulting_data["amount1"].astype(float) / 10**decimals_1
    )
    resulting_data["virtual_liquidity"] = resulting_data["liquidity"].astype(float)
    resulting_data["virtual_liquidity_adj"] = resulting_data["liquidity"].astype(
        float
    ) / (10 ** ((decimals_0 + decimals_1) / 2))
    resulting_data["token_in"] = resulting_data.apply(
        lambda x: "token0" if (x["amount0_adj"] < 0) else "token1", axis=1
    )
    resulting_data["traded_in"] = resulting_data.apply(
        lambda x: -x["amount0_adj"] if (x["amount0_adj"] < 0) else -x["amount1_adj"],
        axis=1,
    ).astype(float)

    return resulting_data

In [25]:
get_amounts(tick, tickLower, tickUpper, liquidity, dec_0, dec_1)

(0, 88425.77342867808)

In [3]:
#decimals

dec_0 = 6
dec_1 = 18

In [4]:
def get_amount0(sqrtA, sqrtB, liquidity, decimals):

    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    amount0 = (liquidity * 2 ** 96 * (sqrtB - sqrtA) / sqrtB / sqrtA) / 10 ** decimals

    return amount0


def get_amount1(sqrtA, sqrtB, liquidity, decimals):

    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    amount1 = liquidity * (sqrtB - sqrtA) / 2 ** 96 / 10 ** decimals

    return amount1

def get_amounts(tick, tickA, tickB, liquidity, decimal0, decimal1):

    sqrt = int(1.0001 ** (tick / 2) * (2 ** 96))
    sqrtA = int(1.0001 ** (tickA / 2) * (2 ** 96))
    sqrtB = int(1.0001 ** (tickB / 2) * (2 ** 96))

    if sqrtA > sqrtB:
        (sqrtA, sqrtB) = (sqrtB, sqrtA)

    if sqrt <= sqrtA:

        amount0 = get_amount0(sqrtA, sqrtB, liquidity, decimal0)
        return amount0, 0

    elif sqrtB > sqrt > sqrtA:
        amount0 = get_amount0(sqrt, sqrtB, liquidity, decimal0)
        amount1 = get_amount1(sqrtA, sqrt, liquidity, decimal1)
        return amount0, amount1

    else:
        amount1 = get_amount1(sqrtA, sqrtB, liquidity, decimal1)
        return 0, amount1
    
    
def amounts_relation(tick, tickA, tickB, decimals0, decimals1):

    sqrt = (1.0001 ** tick / 10 ** (decimals1 - decimals0)) ** (1 / 2)
    sqrtA = (1.0001 ** tickA / 10 ** (decimals1 - decimals0)) ** (1 / 2)
    sqrtB = (1.0001 ** tickB / 10 ** (decimals1 - decimals0)) ** (1 / 2)

    if sqrt == sqrtA or sqrt == sqrtB:
        relation = 0
    #         print("There is 0 tokens on one side")
    else:
        relation = (sqrt - sqrtA) / ((1 / sqrt) - (1 / sqrtB))
    return relation

In [50]:
# Position https://etherscan.io/address/0xc36442b4a4522e871399cd717abdd847ab11fe88#readContract
# TokenId 236595
# Transaction - https://etherscan.io/tx/0x9b173baa9442fcb53cdde9876eaabeac9a71ff93d822e5398b5cfd6a5d38a199

token0 = 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
token1 = 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
fee = 3000
tickLower = 196200
tickUpper = 201660
pos_liquidity = 26938023944267617
feeGrowthInside0LastX128 = 1146045437454411223706317130808195
feeGrowthInside1LastX128 = 472868971286049499087619319314568638950194
tokensOwed0 = 863372171
tokensOwed1 = 362821509667276570

In [6]:
# ticks i_l, i_u info

#Lower
L_liquidityGross = 18216572431750930
L_liquidityNet = 18216572431750930
L_feeGrowthOutside0X128 = 771898924715078716122457993949735
L_feeGrowthOutside1X128 = 299686152818024558401586591746381951788873
L_tickCumulativeOutside = 874561673429
L_secondsPerLiquidityOutsideX128 = 198044337547991777424606226079348331092728
L_secondsOutside = 1624605540
L_initialized = True

#Upper
U_liquidityGross = 1683230461217016778
U_liquidityNet = 1299234642981930080
U_feeGrowthOutside0X128 = 1226433780500214190158000189806691
U_feeGrowthOutside1X128 = 365746732484003642662975858100178521758045
U_tickCumulativeOutside = 4519733975844
U_secondsPerLiquidityOutsideX128 = 198044337547992220998400984601510297703566
U_secondsOutside = 1643365229
U_initialized = True



In [16]:
# Pool state - https://etherscan.io/address/0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8#readContract

sqrtPriceX96 =  1617871362335671645645002850471718
tick = 198495
observationIndex = 976
observationCardinality = 1440
observationCardinalityNext = 1440
feeProtocol = 0

liquidity = 10200180983978284202

feeGrowthGlobal0X128 = 2046477727527315981651672382110787
feeGrowthGlobal1X128 = 724331803708946607581226850148167065366051

In [36]:
(sqrtPriceX96/2**96)**2 * 10**(dec_0-dec_1)

0.0004169930415991001

** To find 

In [14]:
# All in 1 token

In [98]:
# Check upper tick
def get_fees_for_range(tick, tickA, tickB, feeGrowthGlobal, U_feeGrowthOutside, L_feeGrowthOutside):
    if tick >= tickB:
        f_a = feeGrowthGlobal - U_feeGrowthOutside
    else:
        f_a = U_feeGrowthOutside
    
    if tick >= tickA:
        f_b = L_feeGrowthOutside
    else:
        f_b = feeGrowthGlobal - L_feeGrowthOutside

    f_range = feeGrowthGlobal - f_a - f_b
    
    return f_range / 2**128

In [99]:
print(get_fees_for_range(tick, tickLower, tickUpper, feeGrowthGlobal1X128, U_feeGrowthOutside1X128, L_feeGrowthOutside1X128))
print(get_fees_for_range(tick, tickLower, tickUpper, feeGrowthGlobal0X128, U_feeGrowthOutside0X128, L_feeGrowthOutside0X128))

173.0883646421886
1.414855043699903e-07


In [91]:
feeGrowthInside1LastX128/2**128

1389.6370110647442

In [100]:
def fees_uncollected_position(tick, tickA, tickB, feeGrowthGlobal, U_feeGrowthOutside, L_feeGrowthOutside, feeGrowthInside, pos_liquidity, decimals):
    f_range_uncollected = get_fees_for_range(tick, tickA, tickB, feeGrowthGlobal, U_feeGrowthOutside, L_feeGrowthOutside)
    return pos_liquidity * (f_range_uncollected - feeGrowthInside/2**128)

In [101]:
fees_uncollected_position(tick, tickLower, tickUpper, feeGrowthGlobal1X128, U_feeGrowthOutside1X128, L_feeGrowthOutside1X128, feeGrowthInside1LastX128, pos_liquidity, dec_1)



-3.2771416566697157e+19

In [102]:
fees_uncollected_position(tick, tickLower, tickUpper, feeGrowthGlobal0X128, U_feeGrowthOutside0X128, L_feeGrowthOutside0X128, feeGrowthInside0LastX128, pos_liquidity, dec_0)



-86913900179.84258

In [76]:
feeGrowthInside1LastX128 / 2**128

1389.6370110647442

In [103]:
get_amounts(tick,tickLower, tickUpper, pos_liquidity, dec_0, dec_1)

(193076.88153517284, 59.630328294076946)

In [49]:
def to_float(x,e):
    c = abs(x)
    sign = 1 
    if x < 0:
        # convert back from two's complement
        c = x - 1 
        c = ~c
        sign = -1
    f = (1.0 * c) / (2 ** e)
    f = f * sign
    return f

In [60]:
to_float(589288769666640050413360360828235182981321, 128)

1731.764049365379

In [63]:
to_float(1536525621214938325098269939695336, 128)

4.515442969079665e-06

In [2]:
import pickle

In [6]:
with open('data/eth_usdc_swap.pkl', "rb") as fh:
    data = pickle.load(fh)

In [7]:
import pandas as pd

In [8]:
pd.DataFrame(data)

,id,timestamp,tick,amount0,amount1,amountUSD
0,0x000035d5e2bacbcdfad84b1b97113f2c4a83534bc7e4...,1621348552,195353,-341098.711504,104.1156,341744.6271658624201755352726503601
1,0x0000cbd95c55d590cfd68a9c593957aca9de75824e94...,1638835892,192536,-744.608566,0.171619960261869437,745.729040061016820805721675871721
2,0x0000d28bd990d3232d2c72b10ad5b99c2803c4648eaf...,1623239511,198022,26582.946537,-10.541053894510651217,26542.96632908660648949465857012522
3,0x00012ce2df8aacdf8c0ec30631170845704b2249f7b0...,1627390084,198938,200000,-86.928111599666651722,199677.3127029941910943241438634473
4,0x000130477fe48e93859ae71712e540327aa48b18c54f...,1630237508,195615,-1116978.523808,350,1119078.488894339722287168798626164
...,...,...,...,...,...,...
12995,0x0e0ce49275d594edb858c1b14dd0f9028510292a68d1...,1637763822,192886,557850.263723,-132.461619620591735566,556805.023914860124348187291632405
12996,0x0e0d874a7581194f18f6d61ea061de18831c66b96352...,1646414027,197358,157600.45,-58.500015831458282413,157338.818080755112285765395126147
12997,0x0e0db9c4a365760b5716092966a546c49151fe9bcef5...,1649103273,194750,-4650,1.337304308940230188,4657.004947780983065107394209136967
12998,0x0e0e743f4a1a329f9bb493a40c86d07cb8722a8e7a72...,1624859298,200453,-393200.386124,200,393801.5121945686181011115428126334


In [9]:
pd.__version__

'1.4.2'